# In Semantic Kernel 1.0, How Do You Pipeline Functions? This is how!

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.6.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.6.1-alpha"

Installed Packages Microsoft.SemanticKernel, 1.6.1 Microsoft.SemanticKernel.Plugins.Core, 1.6.1-alpha

In [2]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

# Basic Version

This uses the basic Semantic Kernel [prompt template format](https://learn.microsoft.com/en-us/semantic-kernel/prompts/configure-prompts?tabs=Csharp).

In [3]:
#!import config/Pipeliner.cs

Console.WriteLine("================ PIPELINE ================");
{
    // Create a pipeline of functions that will parse a string into an int, multiply it by a double, truncate it to an int, and then humanize it.
    KernelFunction parseInt32 = KernelFunctionFactory.CreateFromMethod((string s) => double.Parse(s, CultureInfo.InvariantCulture), "parseInt32");
    KernelFunction multiplyByN = KernelFunctionFactory.CreateFromMethod((double i, double n) => i * n, "multiplyByN");
    KernelFunction truncate = KernelFunctionFactory.CreateFromMethod((double d) => (int)d, "truncate");

    KernelFunction humanize = KernelFunctionFactory.CreateFromPrompt(new PromptTemplateConfig()
    {
        Template = "Spell out this in English: {{$number}}",
        InputVariables = [new() { Name = "number" }],
    });
    
    KernelFunction pipeline = KernelFunctionPipeliner.Pipe([parseInt32, multiplyByN, truncate, humanize]);

    KernelArguments args = new()
    {
        ["s"] = "123.456",
        ["n"] = (double)78.90,
    };

    // - The parseInt32 function will be invoked, read "123.456" from the arguments, and parse it into (double)123.456.
    // - The multiplyByN function will be invoked, with i=123.456 and n=78.90, and return (double)9740.6784.
    // - The truncate function will be invoked, with d=9740.6784, and return (int)9740, which will be the final result.
    Console.WriteLine(await pipeline.InvokeAsync(kernel, args));
}

================ PIPELINE ================
Nine thousand seven hundred forty.


# Advanced Version

In [5]:
#!import config/Pipeliner.cs

KernelFunction rand = KernelFunctionFactory.CreateFromMethod(() => Random.Shared.Next(), "GetRandomInt32");
KernelFunction mult = KernelFunctionFactory.CreateFromMethod((int i, int j) => i * j, "Multiply");

// - Invokes rand and stores the random number into args["i"]
// - Invokes rand and stores the random number into args["j"]
// - Multiplies arg["i"] and args["j"] to produce the final result
KernelFunction graph = KernelFunctionPipeliner.Pipe(new[]
{
    (rand, "i"),
    (rand, "j"),
    (mult, "")
});

Console.WriteLine(await graph.InvokeAsync(kernel));

-555716488
